In [1]:
!pip install -q -U faiss-cpu langchain langchain-community langchain_huggingface sentence-transformers pypdf streamlit streamlit-lottie streamlit-extras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 473.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!rm myscript.sh
!echo '#!/bin/bash' > myscript.sh
!echo 'ollama serve' >> myscript.sh
!chmod +x myscript.sh
!nohup ./myscript.sh &

rm: cannot remove 'myscript.sh': No such file or directory
nohup: appending output to 'nohup.out'


In [4]:
!nohup ollama run llama3-chatqa > ollama_model_instruct.log 2>&1 &

In [5]:
!ollama list

NAME	ID	SIZE	MODIFIED 


In [6]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2')

llm = Ollama(model="llama3-chatqa:latest")

# Function to load and process documents
def load_documents(file_path: str):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)
    return texts

# Function to create or load FAISS index
def get_faiss_index(texts, index_name="faiss_index"):
    if os.path.exists(f"{index_name}.faiss"):
        vector_store = FAISS.load_local(index_name, embedding_model)
    else:
        vector_store = FAISS.from_documents(texts, embedding_model)
        vector_store.save_local(index_name)
    return vector_store

# Main RAG pipeline
class RAGQASystem:
    def __init__(self):
        self.vector_store = None
        self.qa_chain = None

    def load_documents(self, file_path: str):
        texts = load_documents(file_path)
        self.vector_store = get_faiss_index(texts)
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=self.vector_store.as_retriever(search_kwargs={"k": 5}),
            return_source_documents=True
        )

    def answer_question(self, query: str):
        if not self.qa_chain:
            raise ValueError("Documents not loaded. Please load documents first.")

        result = self.qa_chain.invoke({
            "system": "Strictly follow these instructions: Provide only the exact answer in a short and clear way as stated in the context text in short. If the answer is not present, return 'Answer not found'. Do not generate or include any additional text or information beyond the answer provided in the context.",
            "system": "You will be heavily penalized if you do not follow the above instructions.",
            "query": query
        }, temperature=0.0)

        return result['result'], result['source_documents']

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
rag_system = RAGQASystem()
rag_system.load_documents("India-wikipedia.pdf")

In [9]:
queries = [
    "How many Cricket World Cups has India won and in which year?",
    "Which country is India's largest arms supplier?",
    "What was the worth of Indian economy in 2024 and what was its rank by market exchange rates and purchasing power parity?"
]

for query in queries:
    answer, source_docs = rag_system.answer_question(query)
    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print(f"Souce Documents: {[doc.page_content for doc in source_docs]}")
    print()

Query: How many Cricket World Cups has India won and in which year?
Answer:  India has won two cricket world cups. They have won it in 1983 edition and in 2011.
Souce Documents: ["An examp le of this dominance is the basketball competiti on where the Indian team won four out of five\ntournaments to date.[499][500]\nIndia has hosted or co-hosted several international sporting events: the 1951  and 1982 Asian Games ; the\n1987 , 1996 , 2011 and 2023 ICC Men's Cricket World Cup tournaments (and is also scheduled to host it in\n2031 ); the 1978 , 1997  and 2013 ICC Women's Cricket World Cup tournaments (and is also scheduled to\nhost it in 2025 ); the 1987 , 1985  and 2016 South Asian Games ; the 1990–91 Men's Asia Cup; the 2002\nChess World Cup; the 2003 Afro -Asian Games; the 2006 ICC Cricket Champion's Trophy  (and is also\nscheduled to host it in 2029 ); the 2006 Women's Asia Cup; the 2009 World Badminton Championships ;\nthe 2010 Hockey World Cup; the 2010 Commonwealt h Games ; the 20